In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. Download stock data
ticker = 'AAPL'  # You can change this to any stock symbol
data = yf.download(ticker, start='2010-01-01', end='2025-01-01')

# 2. Prepare the closing prices
closing_prices = data['Close'].values.reshape(-1, 1)

# 3. Normalize the data
scaler = MinMaxScaler()
scaled_prices = scaler.fit_transform(closing_prices)

# 4. Create sequences (60 days to predict next day)
sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_prices)):
    X.append(scaled_prices[i - sequence_length:i, 0])
    y.append(scaled_prices[i, 0])

X = np.array(X)
y = np.array(y)

# Reshape input for LSTM: (samples, time_steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# 5. Split into training and test sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 6. Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

# 7. Compile and train
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# 8. Predict and inverse transform
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# 9. Plot the results
plt.figure(figsize=(14, 5))
plt.plot(actual, color='blue', label='Actual Stock Price')
plt.plot(predictions, color='red', label='Predicted Stock Price')
plt.title(f'{ticker} Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# 10. Predict the next day's stock price
last_60_days = scaled_prices[-60:]
last_60_days = last_60_days.reshape(1, -1, 1)
next_day_prediction = model.predict(last_60_days)
next_day_price = scaler.inverse_transform(next_day_prediction)
print(f"📈 Predicted next day's price for {ticker}: ${next_day_price[0][0]:.2f}")
